In [1]:
import fiftyone as fo
import os
import shutil

# -------------------------
# 1. ตั้งค่า FiftyOne
# -------------------------
config = fo.config
config.database_validation = False

# -------------------------
# 2. กำหนดพาธ
# -------------------------
dataset_dir = r"C:\Users\HP\Downloads\ThaiCharacter Dataset\round2"
dataset_name = "CharRecognitionProject_New"

base_export_dir = r"C:\Users\HP\Downloads\ThaiCharacter Dataset\Separated_Images"
training_dir = r"C:\Users\HP\Downloads\ThaiCharacter Dataset\Final_Training_Set"

# -------------------------
# 3. โหลดหรือสร้าง dataset
# -------------------------
if dataset_name in fo.list_datasets():
    dataset = fo.load_dataset(dataset_name)
    print(f"✅ โหลด dataset ที่มีอยู่: {len(dataset)} รูป")
else:
    dataset = fo.Dataset.from_dir(
        dataset_dir=dataset_dir,
        dataset_type=fo.types.ImageClassificationDirectoryTree,
        name=dataset_name
    )
    print(f"✅ สร้าง dataset ใหม่: {len(dataset)} รูป")

# -------------------------
# 4. สร้าง field 'tags' ถ้ายังไม่มี
# -------------------------
if "tags" not in dataset.get_field_schema():
    dataset.add_sample_field("tags", fo.types.List, embedded_type=fo.types.String)
    print("✅ สร้าง field 'tags' เรียบร้อยแล้ว")

# -------------------------
# 5. เปิด FiftyOne App
# -------------------------
session = fo.launch_app(dataset, port=5153)  # desktop=False สำหรับ notebook

✅ โหลด dataset ที่มีอยู่: 14580 รูป


In [2]:
# 💾 Cell นี้ให้รันหลังจากติดแท็กเสร็จแล้ว
dataset.persistent = True
dataset.save()
print("✅ บันทึกแท็กเรียบร้อยแล้ว!")
print(f"จำนวนรูปที่มีแท็ก: {len(dataset.match_tags('WTF')) + len(dataset.match_tags('hardtosee'))} รูป")

✅ บันทึกแท็กเรียบร้อยแล้ว!
จำนวนรูปที่มีแท็ก: 651 รูป


In [3]:
# -------------------------
# 6. ส่งออกภาพที่ติดแท็ก 'WTF' และ 'hardtosee'
# -------------------------
for tag_name in ["WTF", "hardtosee"]:
    export_dir = os.path.join(base_export_dir, tag_name)
    
    # ลบโฟลเดอร์เก่าทิ้งถ้ามีอยู่
    if os.path.exists(export_dir):
        shutil.rmtree(export_dir)
        print(f"🗑️ ลบโฟลเดอร์ '{tag_name}' เก่าทิ้งเรียบร้อย")
    
    view = dataset.match_tags(tag_name)
    num_samples = len(view)
    if num_samples > 0:
        view.export(
            export_dir=export_dir,
            dataset_type=fo.types.ImageClassificationDirectoryTree,
            label_field="ground_truth",
            export_media=True
        )
        print(f"✅ ส่งออก {num_samples} ภาพที่มีแท็ก '{tag_name}' ไปยัง: {export_dir}")
    else:
        print(f"❌ ไม่พบภาพที่มีแท็ก '{tag_name}'")

# -------------------------
# 7. ส่งออกภาพดี (ไม่มีแท็ก)
# -------------------------
# ลบโฟลเดอร์ training เก่าทิ้งถ้ามีอยู่
if os.path.exists(training_dir):
    shutil.rmtree(training_dir)
    print(f"🗑️ ลบโฟลเดอร์ training เก่าทิ้งเรียบร้อย")

problem_tags_condition = (
    (fo.ViewField("tags").contains("WTF")) |
    (fo.ViewField("tags").contains("hardtosee"))
)
good_samples_view = dataset.match(~problem_tags_condition)
num_good_samples = len(good_samples_view)

if num_good_samples > 0:
    good_samples_view.export(
        export_dir=training_dir,
        dataset_type=fo.types.ImageClassificationDirectoryTree,
        label_field="ground_truth",
        export_media=True
    )
    print(f"✨ ส่งออกภาพดี {num_good_samples} รูป ไปยัง: {training_dir} เรียบร้อยแล้ว!")
else:
    print("⚠️ ไม่พบภาพดีสำหรับ Train")

 100% |█████████████████| 410/410 [1.4s elapsed, 0s remaining, 282.6 samples/s]         
✅ ส่งออก 410 ภาพที่มีแท็ก 'WTF' ไปยัง: C:\Users\HP\Downloads\ThaiCharacter Dataset\Separated_Images\WTF
 100% |█████████████████| 241/241 [799.5ms elapsed, 0s remaining, 301.4 samples/s]      
✅ ส่งออก 241 ภาพที่มีแท็ก 'hardtosee' ไปยัง: C:\Users\HP\Downloads\ThaiCharacter Dataset\Separated_Images\hardtosee
 100% |█████████████| 13929/13929 [47.0s elapsed, 0s remaining, 266.4 samples/s]      
✨ ส่งออกภาพดี 13929 รูป ไปยัง: C:\Users\HP\Downloads\ThaiCharacter Dataset\Final_Training_Set เรียบร้อยแล้ว!


In [9]:
# -------------------------
# 6. ส่งออกภาพที่ติดแท็ก 'WTF' และ 'hardtosee'
# -------------------------
for tag_name in ["WTF", "hardtosee"]:
    export_dir = os.path.join(base_export_dir, tag_name)
    view = dataset.match_tags(tag_name)
    num_samples = len(view)
    if num_samples > 0:
        view.export(
            export_dir=export_dir,
            dataset_type=fo.types.ImageClassificationDirectoryTree,
            label_field="ground_truth",
            export_media=True
        )
        print(f"✅ ส่งออก {num_samples} ภาพที่มีแท็ก '{tag_name}' ไปยัง: {export_dir}")
    else:
        print(f"❌ ไม่พบภาพที่มีแท็ก '{tag_name}'")

# -------------------------
# 7. ส่งออกภาพดี (ไม่มีแท็ก)
# -------------------------
problem_tags_condition = (
    (fo.ViewField("tags").contains("WTF")) |
    (fo.ViewField("tags").contains("hardtosee"))
)
good_samples_view = dataset.match(~problem_tags_condition)
num_good_samples = len(good_samples_view)

if num_good_samples > 0:
    good_samples_view.export(
        export_dir=training_dir,
        dataset_type=fo.types.ImageClassificationDirectoryTree,
        label_field="ground_truth",
        export_media=True
    )
    print(f"✨ ส่งออกภาพดี {num_good_samples} รูป ไปยัง: {training_dir} เรียบร้อยแล้ว!")
else:
    print("⚠️ ไม่พบภาพดีสำหรับ Train")


Directory 'C:\Users\HP\Downloads\ThaiCharacter Dataset\Separated_Images\WTF' already exists; export will be merged with existing files
 100% |█████████████████| 410/410 [1.2s elapsed, 0s remaining, 356.4 samples/s]         
✅ ส่งออก 410 ภาพที่มีแท็ก 'WTF' ไปยัง: C:\Users\HP\Downloads\ThaiCharacter Dataset\Separated_Images\WTF
Directory 'C:\Users\HP\Downloads\ThaiCharacter Dataset\Separated_Images\hardtosee' already exists; export will be merged with existing files
 100% |█████████████████| 241/241 [676.4ms elapsed, 0s remaining, 356.3 samples/s]      
✅ ส่งออก 241 ภาพที่มีแท็ก 'hardtosee' ไปยัง: C:\Users\HP\Downloads\ThaiCharacter Dataset\Separated_Images\hardtosee
Directory 'C:\Users\HP\Downloads\ThaiCharacter Dataset\Final_Training_Set' already exists; export will be merged with existing files
 100% |█████████████| 13929/13929 [38.0s elapsed, 0s remaining, 406.8 samples/s]      
✨ ส่งออกภาพดี 13929 รูป ไปยัง: C:\Users\HP\Downloads\ThaiCharacter Dataset\Final_Training_Set เรียบร้อยแล้